In [2]:
import pandas as pd

In [3]:
train = pd.read_excel('data/train.xlsx', index_col='Id')

In [4]:
train.head()

,Category,Topic,Source,Headline,Text,Link
Id,,,,,,
1,Fake,Education,El Ruinaversal,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...","RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",http://www.elruinaversal.com/2017/06/10/rae-in...
2,Fake,Education,Hay noticia,"La palabra ""haiga"", aceptada por la RAE","La palabra ""haiga"", aceptada por la RAE La Rea...",https://haynoticia.es/la-palabra-haiga-aceptad...
3,Fake,Education,El Ruinaversal,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,http://www.elruinaversal.com/2018/05/06/yordi-...
4,True,Education,EL UNIVERSAL,UNAM capacitará a maestros para aprobar prueba...,UNAM capacitará a maestros para aprobar prueba...,http://www.eluniversal.com.mx/articulo/nacion/...
5,Fake,Education,Lamula,pretenden aprobar libros escolares con conteni...,Alerta: pretenden aprobar libros escolares con...,https://redaccion.lamula.pe/2018/06/19/memoria...


In [5]:
train.shape

(676, 6)

In [10]:
train[train['Category'] == 'Fake']['Source'].value_counts().head(20)

El Dizque                  99
El Ruinaversal             55
Hay noticia                31
Censura 0                  25
La voz popular             16
Retroceso                  12
Modo Noticia               11
El mundo today              6
Modo noticia                5
EL MUNDO TODAY              5
La otra opinion             4
Nación Unida                4
Argumento Político          4
Modonoticia                 3
Lo que calla la TV          2
La Voz Popular              2
Todo Informe                2
HispanTV                    2
Inciclopedia                2
El librero de Gutenberg     1
Name: Source, dtype: int64